<a href="https://colab.research.google.com/github/brikesh987/intro_to_llm_agents/blob/main/LlmaIndex_AgentIntro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Product docoment Q&A Version 2**

Algorithm:

1. Instatiate Pinecone
2. Create an index in pinecone
3. *Upload a simple Pdf and parse using langchain*
4. Choose the chunking mechanism for document. For now using spaCy, later explore the capabilities in langchain
5. Chunk the document
6. Upsert in the pinecone index (vector store)
7. Import langchain module
8. Create a prompt template
9. create a chain - includes (prompt template and llm - OpenAI instance)
10. Create a retriver from the langchain.vector store's pinecone instance
11 Create a new retriver chain - using the retriver and the document chain created in 8
12.Call invoke function of the retrieval_chain and print the response["answer"]

**This is special document that has images as well. We need multi-model vector which can do a semantic search and also retrieve the images**

Following this link to implement that. It uses **ChromaDB**

1. This approach extracts the information from large PDF and the uses LLM to summarize that text
2

[Multi model RAG](https://github.com/langchain-ai/langchain/blob/master/cookbook/Multi_modal_RAG.ipynb)


**Install libraries for chunking**
There are several options libraries like NLTK, spaCy, Langchain
Here using spaCy
Read more here:https://www.pinecone.io/learn/chunking-strategies/

In [ ]:
%pip install langchain unstructured[all-docs] pydantic lxml
%pip install pytesseract
!apt-get install tesseract-ocr
!apt-get install poppler-utils
%pip install pdf2image
%pip install "unstructured[all-docs]" pillow pydantic lxml pillow matplotlib chromadb tiktoken
%pip install -U langchain openai chromadb langchain-experimental # (newest versions required for multi-modal)
%pip install langchain_openai
%pip install langchain[chat-davinci]
%pip install cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 51.6 M

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 32 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,849 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 121747 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
%pip install cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.6 MB/s eta 0:00:00


**Extract the elements from the document**

In [ ]:
import pytesseract
from langchain.text_splitter import CharacterTextSplitter
from unstructured.partition.pdf import partition_pdf


# Extract elements from PDF
def extract_pdf_elements(path, fname):
    """
    Extract images, tables, and chunk text from a PDF file.
    path: File path, which is used to dump images (.jpg)
    fname: File name
    """
    ## HACK to fix a package dependency issue. This is installed at a different location
    pytesseract.pytesseract.tesseract_cmd = (
        r'/usr/bin/tesseract'
    )

    return partition_pdf(
        filename=path + fname,
        extract_images_in_pdf=True,
        infer_table_structure=True,
        chunking_strategy="by_title",
        max_characters=1000,
        new_after_n_chars=990,
        combine_text_under_n_chars=1000,
        image_output_dir_path=path,
    )


# Categorize elements by type
def categorize_elements(raw_pdf_elements):
    """
    Categorize extracted elements from a PDF into tables and texts.
    raw_pdf_elements: List of unstructured.documents.elements
    """
    tables = []
    texts = []
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            tables.append(str(element))
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            texts.append(str(element))
    return texts, tables


# File path
fpath = "/content/sample_data/"

# File name
#fname = "model_y_owners_manual_north_america_en_2020_44.pdf"
fname = "LG_Monitor.pdf"

# Get elements
raw_pdf_elements = extract_pdf_elements(fpath, fname)

# Get text, tables
texts, tables = categorize_elements(raw_pdf_elements)

# Optional: Enforce a specific token size for texts
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000, chunk_overlap=50
)
joined_texts = " ".join(texts)
texts_4k_token = text_splitter.split_text(joined_texts)

In [ ]:
print("Number of texts:", len(texts_4k_token))
print("Number of tables:", len(tables))

print(texts_4k_token[0])



**Trying open source model to summarize text**
OpenAI API calls getting throttled

In [ ]:
def extract_top_named_entity(model_outputs):
    # Sort entities based on scores in descending order
    sorted_entities = sorted(model_outputs, key=lambda x: x['score'], reverse=True)

    # Return the top entity name
    if sorted_entities:
        return sorted_entities[0]['word']
    else:
        return None

In [ ]:
import langchain
import cohere
from transformers import AutoTokenizer, AutoModelForTokenClassification  # For tokenization and NER
from google.colab import userdata
from langchain.prompts import PromptTemplate
# Choose a summarization library:
# Option 1: Cohere (requires API key and access)
# Option 2: Open-source model from Hugging Face
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import pipeline
import torch

# Load tokenizer and NER model
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
ner_model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
nlp = pipeline("ner", model=ner_model, tokenizer=tokenizer)

you_have_cohere_api_key = True
your_cohere_api_key = userdata.get('cohere_key')
# Choose and load summarization model
if you_have_cohere_api_key:
    # Option 1: Cohere
    co = cohere.Client(your_cohere_api_key)
    summarizer = co
else:
    # Option 2: Open-source model
    summarizer = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
    bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

# Define summarization prompt
prompt = PromptTemplate.from_template("Summarize the text, highlighting important entities like {entities}.")

def summarize_text(text):
    # Tokenize text
    """
    tokens = tokenizer(text, return_tensors="pt")
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = tokenizer.encode(text, return_tensors="pt")
    print(input_ids)
    """
    # Resize input tensor if necessary
    """
    max_length = ner_model.config.max_position_embeddings
    if input_ids.size(1) > max_length:
        input_ids = input_ids[:, :max_length]  # Truncate input if too long
    elif input_ids.size(1) < max_length:
        pad_amount = max_length - input_ids.size(1)
        input_ids = torch.nn.functional.pad(input_ids, (0, pad_amount), value=tokenizer.pad_token_id)
    """
    # Run NER
    #entities = ner_model(tokens).entities[0]  # Extract entities for the first chunk
    outputs = nlp(text)
    #entities = extract_top_named_entity(outputs, tokenizer)
    entities = []
    for output in outputs:
        if output['score'] > 0.5:
            entities.append(output['word'])

    # Create prompt with extracted entities
    summarization_prompt = prompt.format(entities=entities)

    # Run summarization model
    if you_have_cohere_api_key:
        response = summarizer.summarize(
                       text=text,
                       model='command',
                       length='long',
                       extractiveness='medium'
        )
        summary = response.summary
    else:
        input_ids = bart_tokenizer(summarize_text=text, prompt=summarization_prompt, return_tensors="pt").input_ids
        summary_ids = summarizer.generate(input_ids)
        summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

# Example usage
#text = "John Smith traveled to Paris with his friend Alice."
text = texts_4k_token[0]
summary = summarize_text(text)
print(summary)


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


LG LED monitors are LCD monitors with LED backlighting. This owner's manual provides information for installing and using the IPS LED Monitor (Model Name: 27UD88). Please read the safety information carefully before using the product. The LG LED monitor has a joystick button power LED that allows you to easily control the functions of the monitor. You can adjust the height and angle of the screen for a comfortable viewing experience. For safety, always use genuine components and follow the provided instructions when moving or installing the monitor to prevent damage or injury.


In [ ]:
# Import libraries
import langchain
from transformers import ...  # For tokenization and NER
from cohere import Cohere  # Or another summarization library

# Load models and initialize chain
tokenizer = ...  # Initialize tokenizer
ner_model = ...  # Load NER model
summarization_model = ...  # Load summarization model

# Define summarization prompt
prompt = langchain.PromptTemplate("Summarize the text, highlighting important entities like {entities}.")

def summarize_text(text):
  # Tokenize text
  tokens = tokenizer(text, return_tensors="pt")

  # Run NER
  entities = ner_model(tokens).entities[0]  # Extract entities for the first chunk

  # Create prompt with extracted entities
  summarization_prompt = prompt.instantiate(entities=entities)

  # Run summarization model
  summary = summarization_model(summarization_prompt)(text)

  return summary

# Example usage
text = "John Smith traveled to Paris with his friend Alice."
summary = summarize_text(text)
print(summary)


**Generate Text Summaries**

In [ ]:
## Invovking the summarization using the langchain is making OpenAI throttle the requests.
## Trying to directly call OpenAI and space out the calls
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from google.colab import userdata
## tring a different model as OpenAI is throttling even 1st request



openai_api_key = userdata.get('openai_api_key')


# Generate summaries of text elements
def generate_text_summaries(texts, tables, summarize_texts=False):
    """
    Summarize text elements
    texts: List of str
    tables: List of str
    summarize_texts: Bool to summarize texts
    """

    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a concise summary of the table or text that is well optimized for retrieval. Table or text: {element} """
    prompt = ChatPromptTemplate.from_template(prompt_text)

    # Text summary chain
    #model = ChatOpenAI(temperature=0, model="gpt-4", openai_api_key=openai_api_key)
    model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", openai_api_key=openai_api_key)

    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    # Initialize empty summaries
    text_summaries = []
    table_summaries = []

    # Apply to text if texts are provided and summarization is requested
    if texts and summarize_texts:
      for input_data in texts:
            result = result = summarize_chain.batch(input_data, {"max_concurrency": 1})
            print(result)
            text_summaries.append(result)
            time.sleep(70)
        #text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})
    elif texts:
        text_summaries = texts

    # Apply to tables if tables are provided
    if tables:
        #table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})
        for input_data in tables:
            result = result = summarize_chain.batch(input_data, {"max_concurrency": 1})
            print(result)
            table_summaries.append(result)
            time.sleep(70)  # Space out the calls with a 2-second interval

    return text_summaries, table_summaries


# Get text, table summaries
text_summaries, table_summaries = generate_text_summaries(
    texts_4k_token, tables, summarize_texts=True
)

In [ ]:
print(len(text_summaries))

**Make Throttled calls to Open AI so avoid Rate Limit _ THIS needs to be fixed**

Trying to use map-reduce for a very large document

[Map Reduce](https://harikirankante.hashnode.dev/how-to-summarize-large-documents-using-langchain-and-openai-in-python)

In [ ]:
# @title
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from google.colab import userdata
from langchain.prompts import PromptTemplate
import time

openai_api_key = userdata.get('openai_api_key')

# Function to call OpenAI summarization with a 2-second interval
def call_openai_summarization(prompt_chain, inputs, max_concurrency=5):
    model = ChatOpenAI(temperature=0, model="gpt-4", openai_api_key=openai_api_key)
    chain =  {"element": lambda x: x} | prompt | model | StrOutputParser()

    results = []
    for input_data in inputs:
       result = chain.batch(input_data, {"max_concurrency": 5})
       results.append(result)
       time.sleep(2)  # Space out the calls with a 2-second interval

    return results

# Generate summaries of text elements
def generate_text_summaries(texts, tables, summarize_texts=False):
    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a concise summary of the table or text that is well optimized for retrieval. Table or text: {element} """

    map_template = """Write a concise summary of the following content:
    {content}
    Summary:
    """

    #prompt = ChatPromptTemplate.from_template(prompt_text)
    prompt = ChatPromptTemplate.from_template(map_template)



    # Initialize empty summaries
    text_summaries = []
    table_summaries = []


    model = ChatOpenAI(temperature=0, model="gpt-4", openai_api_key=openai_api_key)
    #chain =  {"element": lambda x: x} | prompt | model | StrOutputParser()
    chain =  {"content": lambda x: x} | map_template | model | StrOutputParser()

    map_chain = LLMChain(prompt=prompt, llm=chain)

    reduce_template = """The following is set of summaries:
    {doc_summaries}
    Summarize the above summaries with all the key details
    Summary:"""

    reduce_prompt = PromptTemplate.from_template(reduce_template)
    reduce_chain = LLMChain(prompt=reduce_prompt, llm=chain)
    stuff_chain = StuffDocumentsChain(llm_chain=reduce_chain, document_variable_name="doc_summaries")

    stuff_chain = StuffDocumentsChain(llm_chain=reduce_chain, document_variable_name="content")

    results = []
    # Apply to text if texts are provided and summarization is requested
    # Apply to text if texts are provided and summarization is requested
    if texts and summarize_texts:
        for input_data in texts:
            result = result = chain.batch(input_data, {"max_concurrency": 5})
            results.append(result)
            time.sleep(70)  # Space out the calls with a 2-second interval
            #text_summaries = call_openai_summarization(summarize_chain, texts, max_concurrency=5)
    elif texts:
        text_summaries = texts

    # Apply to tables if tables are provided
    if tables:
        #table_summaries = call_openai_summarization(prompt, tables, max_concurrency=5)
        for input_data in tables:
            result = result = chain.batch(input_data, {"max_concurrency": 1})
            results.append(result)
            time.sleep(70)  # Space out the calls with a 2-second interval

    return text_summaries, table_summaries

# Get text, table summaries
text_summaries, table_summaries = generate_text_summaries(
    texts_4k_token, tables, summarize_texts=True
)


**Generate Image summary**

We will use GPT-4V to produce the image summaries.

The API docs here:

We pass base64 encoded images

In [ ]:
import base64
import os
import time

from langchain_core.messages import HumanMessage


def encode_image(image_path):
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def image_summarize(img_base64, prompt):
    """Make image summary"""
    chat = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=1024, openai_api_key=openai_api_key)

    msg = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content


def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for images
    path: Path to list of .jpg files extracted by Unstructured
    """

    # Store base64 encoded images
    img_base64_list = []

    # Store image summaries
    image_summaries = []

    # Prompt
    prompt = """You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Give a concise summary of the image that is well optimized for retrieval."""

    # Apply to images
    for img_file in sorted(os.listdir(path)):
        if img_file.endswith(".jpg"):
            img_path = os.path.join(path, img_file)
            base64_image = encode_image(img_path)
            img_base64_list.append(base64_image)
            image_summaries.append(image_summarize(base64_image, prompt))
            time.sleep(75)  # Space out the calls with a 2-second interval

    return img_base64_list, image_summaries


# Image summaries
img_path = "/content/sample_data/figures/"
img_base64_list, image_summaries = generate_img_summaries(fpath)

In [ ]:
print(len(image_summaries))

**Add to vectorstore**
Add raw docs and doc summaries to Multi Vector Retriever:

Store the raw texts, tables, and images in the docstore.
Store the texts, table summaries, and image summaries in the vectorstore for efficient semantic retrieval.

In [ ]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings


def create_multi_vector_retriever(
    vectorstore, text_summaries, texts, table_summaries, tables, image_summaries, images
):
    """
    Create retriever that indexes summaries, but returns raw images or texts
    """

    # Initialize the storage layer
    store = InMemoryStore()
    id_key = "doc_id"

    # Create the multi-vector retriever
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
    )

    # Helper function to add documents to the vectorstore and docstore
    def add_documents(retriever, doc_summaries, doc_contents):
        doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
        summary_docs = [
            Document(page_content=s, metadata={id_key: doc_ids[i]})
            for i, s in enumerate(doc_summaries)
        ]
        retriever.vectorstore.add_documents(summary_docs)
        retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

    # Add texts, tables, and images
    # Check that text_summaries is not empty before adding
    if text_summaries:
        add_documents(retriever, text_summaries, texts)
    # Check that table_summaries is not empty before adding
    if table_summaries:
        add_documents(retriever, table_summaries, tables)
    # Check that image_summaries is not empty before adding
    if image_summaries:
        add_documents(retriever, image_summaries, images)

    return retriever


# The vectorstore to use to index the summaries
vectorstore = Chroma(
    collection_name="mm_rag_cj_blog", embedding_function=OpenAIEmbeddings()
)

# Create retriever
retriever_multi_vector_img = create_multi_vector_retriever(
    vectorstore,
    text_summaries,
    texts,
    table_summaries,
    tables,
    image_summaries,
    img_base64_list,
)

In [ ]:
import pytesseract
from typing import Any
from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf

# Get elements
pytesseract.pytesseract.tesseract_cmd = (
    r'/usr/bin/tesseract'
)

raw_pdf_elements = partition_pdf(
    filename=path + "LLaVA.pdf",
    # Using pdf format to find embedded image blocks
    extract_images_in_pdf=True,
    # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
    # Titles are any sub-section of the document
    infer_table_structure=True,
    # Post processing to aggregate text once we have the title
    chunking_strategy="by_title",
    # Chunking params to aggregate text blocks
    # Attempt to create a new chunk 3800 chars
    # Attempt to keep chunks > 2000 chars
    # Hard max on chunks
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=path,
)

**Use Multi-vector retrieve**

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from google.colab import userdata

openai_api_key = userdata.get('openai_api_key')


# Prompt
prompt_text = """You are an assistant tasked with summarizing tables and text. \
Give a concise summary of the table or text. Table or text chunk: {element} """
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
model = ChatOpenAI(temperature=0, model="gpt-4", openai_api_key=openai_api_key)
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

# Prompt
prompt_text = """You are an assistant tasked with summarizing tables and text. \
Give a concise summary of the table or text. Table or text chunk: {element} """
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
model = ChatOpenAI(temperature=0, model="gpt-4", openai_api_key=openai_api_key)
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

# Apply to tables
tables = [i.text for i in table_elements]
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})


**Use GPT4 to summarize them and use multi-vector-retrieve**

[Multi Vector Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector#summary)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


# Generate summaries of text elements
def generate_text_summaries(texts, tables, summarize_texts=False):
    """
    Summarize text elements
    texts: List of str
    tables: List of str
    summarize_texts: Bool to summarize texts
    """

    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a concise summary of the table or text that is well optimized for retrieval. Table or text: {element} """
    prompt = ChatPromptTemplate.from_template(prompt_text)

    # Text summary chain
    model = ChatOpenAI(temperature=0, model="gpt-4")
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    # Initialize empty summaries
    text_summaries = []
    table_summaries = []

    # Apply to text if texts are provided and summarization is requested
    if texts and summarize_texts:
        text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})
    elif texts:
        text_summaries = texts

    # Apply to tables if tables are provided
    if tables:
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

    return text_summaries, table_summaries


# Get text, table summaries
text_summaries, table_summaries = generate_text_summaries(
    texts_4k_token, tables, summarize_texts=True
)

**Use GPT4 to produce image summaries**
We pass base64 encoded images

In [ ]:
import base64
import os
from langchain_core.messages import HumanMessage


def encode_image(image_path):
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def image_summarize(img_base64, prompt):
    """Make image summary"""
    chat = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=1024)

    msg = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content


def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for images
    path: Path to list of .jpg files extracted by Unstructured
    """

    # Store base64 encoded images
    img_base64_list = []

    # Store image summaries
    image_summaries = []

    # Prompt
    prompt = """You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Give a concise summary of the image that is well optimized for retrieval."""

    # Apply to images
    for img_file in sorted(os.listdir(path)):
        if img_file.endswith(".jpg"):
            img_path = os.path.join(path, img_file)
            base64_image = encode_image(img_path)
            img_base64_list.append(base64_image)
            image_summaries.append(image_summarize(base64_image, prompt))

    return img_base64_list, image_summaries


# Image summaries
img_base64_list, image_summaries = generate_img_summaries(fpath)

**Add them to vector store**
Store the raw texts, tables, and images in the docstore.
Store the texts, table summaries, and image summaries in the vectorstore for efficient semantic retrieval.

In [ ]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings


def create_multi_vector_retriever(
    vectorstore, text_summaries, texts, table_summaries, tables, image_summaries, images
):
    """
    Create retriever that indexes summaries, but returns raw images or texts
    """

    # Initialize the storage layer
    store = InMemoryStore()
    id_key = "doc_id"

    # Create the multi-vector retriever
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
    )

    # Helper function to add documents to the vectorstore and docstore
    def add_documents(retriever, doc_summaries, doc_contents):
        doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
        summary_docs = [
            Document(page_content=s, metadata={id_key: doc_ids[i]})
            for i, s in enumerate(doc_summaries)
        ]
        retriever.vectorstore.add_documents(summary_docs)
        retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

    # Add texts, tables, and images
    # Check that text_summaries is not empty before adding
    if text_summaries:
        add_documents(retriever, text_summaries, texts)
    # Check that table_summaries is not empty before adding
    if table_summaries:
        add_documents(retriever, table_summaries, tables)
    # Check that image_summaries is not empty before adding
    if image_summaries:
        add_documents(retriever, image_summaries, images)

    return retriever


# The vectorstore to use to index the summaries
vectorstore = Chroma(
    collection_name="mm_rag_cj_blog", embedding_function=OpenAIEmbeddings()
)

# Create retriever
retriever_multi_vector_img = create_multi_vector_retriever(
    vectorstore,
    text_summaries,
    texts,
    table_summaries,
    tables,
    image_summaries,
    img_base64_list,
)

**Build a Retriever (RAG)**
We need to bin the retrieved doc(s) into the correct parts of the GPT-4V prompt template.

In [ ]:
import io
import re

from IPython.display import HTML, display
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from PIL import Image


def plt_img_base64(img_base64):
    """Disply base64 encoded string as image"""
    # Create an HTML img tag with the base64 string as the source
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    # Display the image by rendering the HTML
    display(HTML(image_html))


def looks_like_base64(sb):
    """Check if the string looks like base64"""
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$", sb) is not None


def is_image_data(b64data):
    """
    Check if the base64 data is an image by looking at the start of the data
    """
    image_signatures = {
        b"\xFF\xD8\xFF": "jpg",
        b"\x89\x50\x4E\x47\x0D\x0A\x1A\x0A": "png",
        b"\x47\x49\x46\x38": "gif",
        b"\x52\x49\x46\x46": "webp",
    }
    try:
        header = base64.b64decode(b64data)[:8]  # Decode and get the first 8 bytes
        for sig, format in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False


def resize_base64_image(base64_string, size=(128, 128)):
    """
    Resize an image encoded as a Base64 string
    """
    # Decode the Base64 string
    img_data = base64.b64decode(base64_string)
    img = Image.open(io.BytesIO(img_data))

    # Resize the image
    resized_img = img.resize(size, Image.LANCZOS)

    # Save the resized image to a bytes buffer
    buffered = io.BytesIO()
    resized_img.save(buffered, format=img.format)

    # Encode the resized image to Base64
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


def split_image_text_types(docs):
    """
    Split base64-encoded images and texts
    """
    b64_images = []
    texts = []
    for doc in docs:
        # Check if the document is of type Document and extract page_content if so
        if isinstance(doc, Document):
            doc = doc.page_content
        if looks_like_base64(doc) and is_image_data(doc):
            doc = resize_base64_image(doc, size=(1300, 600))
            b64_images.append(doc)
        else:
            texts.append(doc)
    return {"images": b64_images, "texts": texts}


def img_prompt_func(data_dict):
    """
    Join the context into a single string
    """
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

    # Adding image(s) to the messages if present
    if data_dict["context"]["images"]:
        for image in data_dict["context"]["images"]:
            image_message = {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            messages.append(image_message)

    # Adding the text for analysis
    text_message = {
        "type": "text",
        "text": (
            "You are financial analyst tasking with providing investment advice.\n"
            "You will be given a mixed of text, tables, and image(s) usually of charts or graphs.\n"
            "Use this information to provide investment advice related to the user question. \n"
            f"User-provided question: {data_dict['question']}\n\n"
            "Text and / or tables:\n"
            f"{formatted_texts}"
        ),
    }
    messages.append(text_message)
    return [HumanMessage(content=messages)]


def multi_modal_rag_chain(retriever):
    """
    Multi-modal RAG chain
    """

    # Multi-modal LLM
    model = ChatOpenAI(temperature=0, model="gpt-4-vision-preview", max_tokens=1024)

    # RAG pipeline
    chain = (
        {
            "context": retriever | RunnableLambda(split_image_text_types),
            "question": RunnablePassthrough(),
        }
        | RunnableLambda(img_prompt_func)
        | model
        | StrOutputParser()
    )

    return chain


# Create RAG chain
chain_multimodal_rag = multi_modal_rag_chain(retriever_multi_vector_img)